In [1]:
import urllib.request

In [2]:
target_url="https://d18ky98rnyall9.cloudfront.net/_fe8d0202cd20a808db6a4d5d06be62f4_clustering_big.txt?Expires=1483142400&Signature=crSq5TkiU2WVkDO-z-rzRXrD48~xff~5dsvuZVGKYed8gP1cAYVkUHpxr5Mulb-mboNnKCz2p8vjC~Se0HYTvuKe7-mxer-xU-WTgEONILlm7jglVmUGoJjDP34ptUxAziRJop0-TWqYTays2wHUrpBEVy4kkVkTbog5bLZGXKo_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"
data=urllib.request.urlopen(target_url).read()
file=open("input_2","w")
file.write(str(data,"utf-8"))
file.close()

In [3]:
def HammingDist(v1,v2):
    if len(v1)!=len(v2):
        raise ValueError("undified for sequence of unequal length")
    else:
        return sum(el1 != el2 for el1, el2 in zip(v1,v2))

In [4]:
graph=[]
with open("input_2") as file:
    for line in file.readlines()[1:]:
        graph.append(line.strip().replace(" ",""))


In [5]:
#use the idea in the 2 sum problem. Here we want to find out pairs with distance =1 and 2. Form a hash table with key
#the node bits and element the label of the node. loop through the nodes and check whether the element that differs by
#1 or 2 is in the hash table.

def diff_1(v):
    candidates=[]
    ori_elements=list(v)
    for i in range(len(ori_elements)):
        cand=ori_elements.copy()
        if cand[i]=="0":
            cand[i]="1"
        else:
            cand[i]="0"
        
        candidates.append("".join(cand))
    return candidates

def diff_2(v):
    candidates=[]
    ori_elements=list(v)
    for i in range(len(ori_elements)):
        for j in range(i+1,len(ori_elements)):
            cand=ori_elements.copy()
            if cand[i]=="0":
                cand[i]="1"
            else:
                cand[i]="0"
            if cand[j]=="0":
                cand[j]="1"
            else:
                cand[j]="0"
            
            candidates.append("".join(cand))
    return candidates

def find(x,tree):
    cache=[]
    child=x
    parent=tree[x]
    
    cache.append(child)
    cache.append(parent)
    
    if parent==child:
        return child
    
    cache.append(child)
    cache.append(parent)
    while parent != child:
        child=parent
        parent=tree[child]
        cache.append(child)
        
    for item in cache:                  #path compression to speed up the algo
        tree[item]=parent
        
    return parent

def union(x,y,tree,rank):
    parent_x=find(x,tree)
    parent_y=find(y,tree)
    if parent_x==parent_y:
        pass
    else:
        if rank[parent_x]>rank[parent_y]:
            tree[parent_y]=parent_x
        else:
            tree[parent_x]=parent_y
            if rank[parent_x]==rank[parent_y]:
                rank[parent_y]+=1

In [6]:
unique_graph=list(set(graph))
graph_dict={}
for i, node in enumerate(unique_graph):
    graph_dict[node]=i

node_diff_1=[]
for i,node1 in enumerate(unique_graph):
    candidate=diff_1(node1)
    for node2 in candidate:
        if node2 in graph_dict:
            j=graph_dict[node2]
            if j > i:                      #avoid counting the pair twice
                node_diff_1.append((i, j))
len(node_diff_1)
            

28231

In [7]:
node_diff_2=[]
for i,node1 in enumerate(unique_graph):
    candidate=diff_2(node1)
    for node2 in candidate:
        if node2 in graph_dict:
            j=graph_dict[node2]
            if j>i:
                node_diff_2.append((i,j))
len(node_diff_2)

324937

In [8]:
N=len(unique_graph)                    #number of unique nodes
node_pairs=node_diff_1+node_diff_2     #node pairs with cost smaller than 3 (1 and 2)

tree=[]                                #initialize tree
for i in range(N):
    tree.append(i)
rank=[0]*N                             #initialize rank

count=0                                #keep track of the number of fusions
for pair in node_pairs:
    v1=pair[0]
    v2=pair[1]
    if find(v1,tree) != find(v2,tree):
        union(v1,v2,tree,rank)
        count+=1
    
N-count                                # number of the clusters

6118